In [9]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}
{-# LANGUAGE TypeSynonymInstances #-}
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE RankNTypes #-}

import Data.Bits             (Bits (bit, complement, popCount, shiftR, (.&.), (.|.)),
                              FiniteBits (finiteBitSize))
import Data.ByteArray.Hash   (FnvHash32 (..), fnv1Hash)
import Data.ByteString.Char8 (pack)
import Data.Char             (intToDigit)
import Data.Semigroup        ((<>))
import Data.Vector           (Vector, drop, singleton, take, (!), (//))
import Data.Word             (Word16, Word32)
import Numeric               (showIntAtBase)
import Prelude               hiding (drop, lookup, take)
import System.TimeIt         (timeIt)
import Text.Show.Pretty      (pPrint)

In [10]:
newtype Binary a = Binary a
    deriving (Enum, Ord, Real, Integral, Eq, Num, Bits, FiniteBits)

instance (FiniteBits a, Show a, Integral a) => Show (Binary a) where
    show (Binary n) = let
        str = showIntAtBase 2 intToDigit n ""
        size = finiteBitSize n
        in replicate (size - length str) '0' <> str

In [11]:
type Hash = Binary Word16

class Hashable a where
    hash :: a -> Hash

In [12]:
instance Hashable String where
    hash s = let
        FnvHash32 h = fnv1Hash (pack s)
        in Binary (fromIntegral h)

In [13]:
data HashBinaryMappedTreeData key value = HashBinaryMappedTreeData Hash key value deriving (Eq, Show)

data HashBinaryMappedTree key value
    = HashBinaryMappedTreeLeaf
    | HashBinaryMappedTreeNode
        (HashBinaryMappedTree key value)
        (Maybe (HashBinaryMappedTreeData key value))
        (HashBinaryMappedTree key value)
    deriving (Eq, Show)

In [14]:
class Mapping mapping where
    empty :: forall k v. mapping k v
    lookup :: forall k v. k -> mapping k v -> Maybe v
    insert :: forall k v. k -> v -> mapping k v -> mapping k v

In [15]:
emptyHashBinaryMappedTree = HashBinaryMappedTreeNode
    HashBinaryMappedTreeLeaf
    Nothing
    HashBinaryMappedTreeLeaf

In [16]:
insertHashBinaryMappedTree :: Hash -> key -> value -> HashBinaryMappedTree key value -> HashBinaryMappedTree key value
insertHashBinaryMappedTree hash key value HashBinaryMappedTreeLeaf = HashBinaryMappedTreeNode
    HashBinaryMappedTreeLeaf
    (Just (HashBinaryMappedTreeData hash key value))
    HashBinaryMappedTreeLeaf
insertHashBinaryMappedTree hash key value leaf@(HashBinaryMappedTreeLeaf leafHash leafKey leafValue)
    | hash == leafHash = HashBinaryMappedTreeLeaf hash key value
    | otherwise = let
        newRootNode = HashBinaryMappedTreeNode HashBinaryMappedTreeNone HashBinaryMappedTreeNone
        insertedExisting = insertHashBinaryMappedTree

: 

In [17]:
:r